In [2]:
from cookie_clusters import *
from selection_pixels_dtw import *
import os
import cv2
from PIL import Image
import rasterio as rio
import re
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [3]:
pixels_de_interet = pix_danone + pix_agri + pix_ensta + pix_apt + pix_lac + pix_foret

In [4]:
dir = "../../ressources/images"
images_list = os.listdir(dir)

images_2A = list()
images_2B = list()
prog = re.compile(r'\w+2A')
for image in images_list:
    if prog.match(image):
        images_2A.append(image)
    else:
        images_2B.append(image)
        
images_2A.sort()
images_2B.sort()
all_images = images_2A + images_2B
all_images = sorted(all_images, key=lambda date: date[16:24])  

In [5]:
list_to_use = all_images
line = 0
matrice = np.zeros((len(pixels_de_interet), len(list_to_use)))
for image in list_to_use:
    with rio.open(dir+'/'+image, 'r') as ds:
        # arr = ds.read()
        band1 = ds.read(1)
        band2 = ds.read(2)
        # band3 = ds.read(3)
        # band4 = raster.read(4)
        band_NDVI = (band1.astype(float) - band2.astype(float))/(band1 + band2)
    # Extraction of target pixels
    for pixel,j in zip(pixels_de_interet, range(len(pixels_de_interet)-1)):
        matrice[j,line] = band_NDVI[pixel[0], pixel[1]]
    line +=1   

/tmp/ipykernel_1123/673945138.py:11: RuntimeWarning: invalid value encountered in divide
  band_NDVI = (band1.astype(float) - band2.astype(float))/(band1 + band2)


In [14]:
kmeans = KMeans(n_clusters=6)
yhat = kmeans.fit_predict(matrice)

/home/afonso/miniconda3/envs/filRouge/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [15]:
import collections
def affichage(yhat,pix_interet):
    name = ''
    dico = collections.Counter(yhat)
    for key in list(dico.keys()):
        dico[key] = [f'number of vectors = {dico[key]}'] 
        for index,pos in zip(yhat,range(len(yhat))):
            if index == key:
                if 0<=pos<=8: name = 'pix_danone'
                elif 9<=pos<=17: name = 'pix_agri'
                elif 18<=pos<=26: name = 'pix_ensta'
                elif 27<=pos<=35: name = 'pix_apt'
                elif 36<=pos<=44: name = 'pix_lac'
                elif 45<=pos<=53: name = 'pix_foret'

                dico[key].append(f'{pix_interet[pos]}:{name}')

    for key in dico:
        print(f'cluster numero {key}:\n-------------------------------')
        for part in dico[key]:
            print(f'{part}')
        print('-------------------------------')

In [16]:
affichage(yhat,pixels_de_interet)

cluster numero 1:
-------------------------------
number of vectors = 9
[383, 497]:pix_danone
[383, 500]:pix_danone
[387, 501]:pix_danone
[383, 504]:pix_danone
[387, 505]:pix_danone
[384, 508]:pix_danone
[388, 509]:pix_danone
[384, 504]:pix_danone
[386, 504]:pix_danone
-------------------------------
cluster numero 2:
-------------------------------
number of vectors = 9
[318, 438]:pix_agri
[322, 435]:pix_agri
[324, 433]:pix_agri
[329, 429]:pix_agri
[333, 426]:pix_agri
[337, 424]:pix_agri
[339, 422]:pix_agri
[344, 418]:pix_agri
[350, 414]:pix_agri
-------------------------------
cluster numero 5:
-------------------------------
number of vectors = 5
[447, 618]:pix_ensta
[433, 617]:pix_ensta
[408, 444]:pix_apt
[412, 463]:pix_apt
[407, 465]:pix_apt
-------------------------------
cluster numero 0:
-------------------------------
number of vectors = 17
[454, 627]:pix_ensta
[454, 631]:pix_ensta
[457, 632]:pix_ensta
[459, 625]:pix_ensta
[450, 641]:pix_ensta
[443, 636]:pix_ensta
[439, 629]:p